In [ ]:
# tensorflow-gpu

In [11]:
# 모델 클래스 생성

class Model :
    
    def __init__(self, sess, name) :
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self) :
        with tf.variable_scope(self.name) :

            # dropout rate
            # 0.7 ~ 0.5 on training, but should be 1 for testing
            # self.keep_prob = tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)

            # input placeholders
            self.X = tf.placeholder(tf.float32, shape = (None, RESIZED_IMAGE[0], RESIZED_IMAGE[1], 1))
            self.y = tf.placeholder(tf.float32, shape = (None, N_CLASSES))

            # Layer1
            conv1 = tf.layers.conv2d(inputs = self.X, filters = 32, kernel_size = [3, 3], padding = 'SAME', activation = tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2, 2], padding = 'SAME', strides = 2)
            dropout1 = tf.layers.dropout(inputs = pool1, rate = 0.3, training = self.training)

            # Layer2
            conv2 = tf.layers.conv2d(inputs = dropout1, filters = 64, kernel_size = [3, 3], padding = 'SAME', activation = tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = [2, 2], padding = 'SAME', strides = 2)
            dropout2 = tf.layers.dropout(inputs = pool2, rate = 0.3, training = self.training)

            # Layer3
            conv3 = tf.layers.conv2d(inputs = dropout2, filters = 128, kernel_size = [3, 3], padding = 'SAME', activation = tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs = conv3, pool_size = [2, 2], padding = 'SAME', strides = 2)
            dropout3 = tf.layers.dropout(inputs = pool3, rate = 0.3, training = self.training)

            # Dense Layer with Relu
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs = flat, units = 625, activation = tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs = dense4, rate = 0.5, training = self.training)

            # Logits (no activation) Layer
            self.logits = tf.layers.dense(inputs = dropout4, units = N_CLASSES)
            # self.sm_logits = tf.nn.softmax(self.logits)

            # define cost/loss & optimizer
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.y))
            self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)

            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            
    def train(self, X_data, y_data, training = True) :
        return self.sess.run([self.cost, self.optimizer], feed_dict = {self.X : X_data, self.y : y_data, self.training : training})
            
    def predict(self, X_test, training = False) :
        return self.sess.run(self.logits, feed_dict = {self.X : X_test, self.training : training})
        
    def get_accuracy(self, X_test, y_test, training = False) :
        return self.sess.run(self.accuracy, feed_dict = {self.X : X_test, self.y : y_test, self.training : training})

### 실시간 영상에 분류 결과 나타내기

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
config = ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

#### 모델 불러오기

In [7]:
CHECK_POINT_DIR = './model.ckpt'

In [8]:
checkpoint = tf.train.get_checkpoint_state(CHECK_POINT_DIR)

In [9]:
print(checkpoint)
print(checkpoint.model_checkpoint_path)

model_checkpoint_path: "./model.ckpt\\model"
all_model_checkpoint_paths: "./model.ckpt\\model"

./model.ckpt\model


In [10]:
# load a trained model
if checkpoint and checkpoint.model_checkpoint_path :
    try : 
        saver = tf.train.import_meta_graph(checkpoint.model_checkpoint_path + '.meta')
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print('Successfully loaded : ', checkpoint.model_checkpoint_path)
    except :
        print("Error on loading old network weights")
else :
    print("Could not find old network weights")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model.ckpt\model
Successfully loaded :  ./model.ckpt\model
